In [ ]:
import pandas as pd
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary

In [ ]:
train_df = pd.read_csv('../dataset/preprocessed_train.csv')
test_df = pd.read_csv('../dataset/preprocessed_test.csv')

In [ ]:
corpus = pd.concat([train_df.comment_text, test_df.comment_text]).reset_index(drop=True).dropna().astype(str)

In [ ]:
corpus_split = corpus.str.split()

In [ ]:
from gensim import corpora, models

dictionary = corpora.Dictionary(corpus_split)

In [ ]:
corpus_BoW = [dictionary.doc2bow(text) for text in corpus_split]


In [ ]:
lda = LdaModel(corpus_BoW, id2word=dictionary, num_topics=50)

In [1]:
import pyLDAvis

In [ ]:
pyLDAvis.gensim.prepare()